In [ ]:
%%bash
# python3 -m pip install -q observable_jupyter==0.1.10
%%capture
# Installation and Imports 
python -m pip install -q fsspec
python -m pip install -q gcsfs

python -m pip install -q tifffile
python -m pip install -q pandas
python -m pip install -q opencv-python
python -m pip install -q h5py
python -m pip install -q matplotlib


python -m pip install -q observable_jupyter==0.1.10

In [ ]:

# Initial Imports

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from copy import deepcopy
from observable_jupyter import embed
from math import cos, sin



import tifffile
import cv2
import h5py
from IPython.display import display


In [ ]:
# Path to Data
base_path = '/Volumes/Samsung_T5/5c/'
region = 'region_0/'
slice_num = 2
replicate_num = 1

# dataset_name = 'Slice' + str(slice_num) + '/Replicate' + str(replicate_num) + '/'
# dataset_suffix = '_S' + str(slice_num) + 'R' + str(replicate_num)
z_index_number = 0
fov = 783

# load transformation matrix 
# This is used to transfer from global coordinates (the coordinates of transcripts 
# and cells across the sample) to mosaic coordinates (used for the large TIF image files).

filename = base_path + region + 'images/micron_to_mosaic_pixel_transform.csv'
transformation_matrix = pd.read_csv(filename, header=None, sep=' ').values


In [ ]:
# %%capture

# load transformation matrix 
# This is used to transfer from global coordinates (the coordinates of transcripts 
# and cells across the sample) to mosaic coordinates (used for the large TIF image files).
# filename = base_path + dataset_name + 'images/micron_to_mosaic_pixel_transform.csv'
# transformation_matrix = pd.read_csv(filename, header=None, sep=' ').values

# load cell boundaries for a single fov
fov = 0

filename = base_path + region + "cell_boundaries/" +'feature_data_' +  str(fov) + '.hdf5'
print(filename)
cellBoundaries = h5py.File(filename)
meta_cell = pd.read_csv(base_path + region + 'cell_metadata' + '.csv', index_col=0)
meta_cell = meta_cell[meta_cell.fov == fov]

z_index = 'zIndex_' + str(z_index_number)
# collect boundaries in fov 
currentCells = []
for inst_cell in meta_cell.index.tolist():
    try:
        # cellBoundaries['featuredata'][inst_cell][z_index]['p_0']['coordinates'][0]
        temp = cellBoundaries['featuredata'][inst_cell][z_index]['p_0']['coordinates'][0]
        boundaryPolygon = np.ones((temp.shape[0], temp.shape[1]+1))
        boundaryPolygon[:, :-1] = temp
        transformedBoundary = np.matmul(transformation_matrix, np.transpose(boundaryPolygon))[:-1]
        currentCells.append(transformedBoundary)
    except:
      pass

minCoord = np.min([np.min(x, axis=1) for x in currentCells], axis=0).astype(int)
maxCoord = np.max([np.max(x, axis=1) for x in currentCells], axis=0).astype(int)      

In [ ]:
# load transcript data - this takes about 25min
transcripts = pd.read_csv(base_path + region + 'detected_transcripts' + '.csv', index_col=0)

In [ ]:
transcripts['gene'].value_counts().head(20)



In [ ]:
# Set gene colors
# colors from https://matplotlib.org/stable/gallery/color/named_colors.html
gene_colors = {
    'Slc1a2': 'teal',
    'Map1b': 'orangered',
    'Dpysl2': 'indigo',
}

In [ ]:
%%capture
# code for compressing data for visuals
import zlib, json, base64
def json_zip(j):
    zip_json_string = base64.b64encode(
        zlib.compress(
            json.dumps(j).encode('utf-8')
        )
    ).decode('ascii')
    return zip_json_string

# process data for interactive visual
keep_genes = gene_colors.keys()
transcripts_keep = transcripts[transcripts.gene.isin(keep_genes)]
transcripts_keep['-global_y'] = -transcripts_keep['global_y']

# rotate the mouse brain to the upright position
theta = np.deg2rad(-195) # -15
rot = np.array([[cos(theta), -sin(theta)], [sin(theta), cos(theta)]])
transcripts_keep[['global_x', '-global_y']] = transcripts_keep[['global_x', '-global_y']].dot(rot)

transcripts_keep['name'] = transcripts_keep['gene']
transcripts_keep['position'] = transcripts_keep[['global_x', '-global_y']].round(2).values.tolist()
transcripts_viz = deepcopy(transcripts_keep[['name', 'position']])
transcripts_string_zip = json_zip(transcripts_viz.to_dict('records'))

transcripts_inputs = {
          'zip_string': transcripts_string_zip,
          'radius_min_pixels': 1.0,
          'height': 800, 
          'gene_colors': gene_colors, 
          'min_zoom': -6
      }

In [ ]:
embed('@vizgen/gene-transcripts-jupyter-v0-2-0', cells=['dashboard'], inputs=transcripts_inputs, display_logo=False)